# Grouping, Splitting and Pivoting Data

In this notebook we'll go over ways to group, split and pivot data using the DataFrame#group_by and and DataFrame#pivot_table functions.

In [1]:
require 'daru'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"http://d3js.org/d3.v3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    window.dispatchEvent(event);\n\t

true

Lets first create a hierarchically indexed DataFrame.

In [2]:
# Create a multi-indexed DataFrame

tuples = [
  [:a,:one,:bar],
  [:a,:one,:baz],
  [:a,:two,:bar],
  [:a,:two,:baz],
  [:b,:one,:bar],
  [:b,:two,:bar],
  [:b,:two,:baz],
  [:b,:one,:foo],
  [:c,:one,:bar],
  [:c,:one,:baz],
  [:c,:two,:foo],
  [:c,:two,:bar]
]
multi_index = Daru::MultiIndex.new(tuples)

vector_arry1 = [11,12,13,14,11,12,13,14,11,12,13,14]
vector_arry2 = [1,2,3,4,1,2,3,4,1,2,3,4]

order_mi = Daru::MultiIndex.new([
  [:a,:one,:bar],
  [:a,:two,:baz],
  [:b,:two,:foo],
  [:b,:one,:foo]])

df_mi = Daru::DataFrame.new([
  vector_arry1, 
  vector_arry2, 
  vector_arry1, 
  vector_arry2], order: order_mi, index: multi_index)

,"[:a, :one, :bar]","[:a, :two, :baz]","[:b, :two, :foo]","[:b, :one, :foo]"
"[:a, :one, :bar]",11,1,11,1
"[:a, :one, :baz]",12,2,12,2
"[:a, :two, :bar]",13,3,13,3
"[:a, :two, :baz]",14,4,14,4
"[:b, :one, :bar]",11,1,11,1
"[:b, :two, :bar]",12,2,12,2
"[:b, :two, :baz]",13,3,13,3
"[:b, :one, :foo]",14,4,14,4
"[:c, :one, :bar]",11,1,11,1
"[:c, :one, :baz]",12,2,12,2


To select a row from a multi-indexed dataframe, you should pass the full tuple to the `#row[]` method. Partial tuples will return whatever row that match the tuple partially.

In [3]:
# Specify complete tuple to choose a single row
df_mi.row[:a, :one,:bar]

#<Daru::Vector:88406950 @name = 0 @size = 4 >
                                0
[:a, :one, :bar]               11
[:a, :two, :baz]                1
[:b, :two, :foo]               11
[:b, :one, :foo]                1

In [4]:
# Specify partial tuple to select index hierarchially
df_mi.row[:a]

,"[:a, :one, :bar]","[:a, :two, :baz]","[:b, :two, :foo]","[:b, :one, :foo]"
"[:one, :bar]",11,1,11,1
"[:one, :baz]",12,2,12,2
"[:two, :bar]",13,3,13,3
"[:two, :baz]",14,4,14,4


## Grouping

The DataFrame#group_by method allows you to group elements in the dataframe by name. This is similar to SQL GROUP BY.

In [5]:
# See grouped rows with the 'groups' method

df = Daru::DataFrame.new({
  a: %w{foo bar foo bar   foo bar foo foo},
  b: %w{one one two three two two one three},
  c:   [1  ,2  ,3  ,1    ,3  ,6  ,3  ,8],
  d:   [11 ,22 ,33 ,44   ,55 ,66 ,77 ,88]
})

# Pass the vectors that are to be grouped in an Array to the group_by method. This 
# will return a Daru::Core::GroupBy object.
grouped = df.group_by([:a, :b])

# See the groups created using the 'groups' method.
grouped.groups

{["bar", "one"]=>[1], ["bar", "three"]=>[3], ["bar", "two"]=>[5], ["foo", "one"]=>[0, 6], ["foo", "three"]=>[7], ["foo", "two"]=>[2, 4]}

In [6]:
# First group by the columns :a and :b and then calculate mean of the grouped rows.
grouped.mean

,c,d
"[:bar, :one]",2,22
"[:bar, :three]",1,44
"[:bar, :two]",6,66
"[:foo, :one]",2.0,44.0
"[:foo, :three]",8,88
"[:foo, :two]",3.0,44.0


The #get_group method can be used for accessing a particualar group(s).

In [7]:
grouped.get_group(["foo", "one"])

,a,b,c,d
0,foo,one,1,11
6,foo,one,3,77


## Pivoting Data

Similar to Excel's Pivot Table, DataFrame provides the #pivot_table functions for quickly pivoting data around a particular value(s) and getting quick insights.

Lets demonstrate using some sales data.

In [8]:
sales = Daru::DataFrame.from_csv 'sales-funnel.csv'

,Account,Manager,Name,Price,Product,Quantity,Rep,Status
0,714466,Debra Henley,Trantow-Barrows,30000,CPU,1,Craig Booker,presented
1,714466,Debra Henley,Trantow-Barrows,10000,Software,1,Craig Booker,presented
2,714466,Debra Henley,Trantow-Barrows,5000,Maintenance,2,Craig Booker,pending
3,737550,Debra Henley,"Fritsch, Russel and Anderson",35000,CPU,1,Craig Booker,declined
4,146832,Debra Henley,Kiehn-Spinka,65000,CPU,2,Daniel Hilton,won
5,218895,Debra Henley,Kulas Inc,40000,CPU,2,Daniel Hilton,pending
6,218895,Debra Henley,Kulas Inc,10000,Software,1,Daniel Hilton,presented
7,412290,Debra Henley,Jerde-Hilpert,5000,Maintenance,2,John Smith,pending
8,740150,Debra Henley,Barton LLC,35000,CPU,1,John Smith,declined
9,141962,Fred Anderson,Herman LLC,65000,CPU,2,Cedric Moss,won


The #pivot_table method accepts an option :index, in which you can specify what vectors you want to index your DataFrame against.

In [9]:
sales.pivot_table index: [:Manager, :Rep]

,Account,Price,Quantity
"[:""Debra Henley"", :""Craig Booker""]",720237.0,20000.0,1.25
"[:""Debra Henley"", :""Daniel Hilton""]",194874.0,38333.333333333336,1.6666666666666667
"[:""Debra Henley"", :""John Smith""]",576220.0,20000.0,1.5
"[:""Fred Anderson"", :""Cedric Moss""]",196016.5,27500.0,1.25
"[:""Fred Anderson"", :""Wendy Yule""]",614061.5,44250.0,3.0


You can also specify the `:values` option for specifying which Vector is to be used for the values.

The `:vectors` options lets you specify the columns to pivot against.

The `:agg` option specifies the aggregation function. This can be any stats method like :mean, :median, :product, etc.

In [10]:
sales.pivot_table(index: [:Manager,:Rep], values: :Price, vectors: [:Product], agg: :sum)

,"[:Price, :CPU]","[:Price, :Software]","[:Price, :Maintenance]","[:Price, :Monitor]"
"[:""Debra Henley"", :""Craig Booker""]",65000,10000,5000,
"[:""Debra Henley"", :""Daniel Hilton""]",105000,10000,,
"[:""Debra Henley"", :""John Smith""]",35000,,5000,
"[:""Fred Anderson"", :""Cedric Moss""]",95000,10000,5000,
"[:""Fred Anderson"", :""Wendy Yule""]",165000,,7000,5000
